将原始文本数据转换为一致性和格式规范的输入数据，为模型训练提供高质量的输入。减少数据噪声，提高模型的泛化能力。
- 去除标点符号
- 统一字母大小写
- 处理数字和特殊字符
- 去除多余空格等

In [15]:
import re
import pandas as pd
from zhon.hanzi import punctuation as zh_punctuation

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [16]:
data = {
    "text": [
        "Hello world! This is an example text with numbers 123 and symbols #, @, and so on.   ",
        "Here's another sentence: it contains HTML tags like <html> and & symbols.",
        "Numbers, punctuations...and STOP words are in this sentence!",
        "这是一段包含HTML标签的文本。<html>标签内容</html>此外，文本中还有特殊字符：#、@、$、¥以及一些无用的信息。",
        "这是一段含有停用词的文本。停用词包括：和、是、的、了、不等。",
        " 此文本包含了前后空格以及多余的空白      间隔。      ",
        "此文本中包含英文标点符号，例如：! @ # $，需要对这些符号进行处理。"
    ]
}

df = pd.DataFrame(data)
stop_words = set(["和", "是", "的", "了", "不", "等",])
stop_words

{'不', '了', '和', '是', '的', '等'}

In [22]:
'此文本中包含英文标点符号'.split()

['此文本中包含英文标点符号']

In [ ]:
def preprocess_text(text: str) -> str:
    # 1. 去首尾空格 + 小写化（对英文有效，对中文无影响）
    text = text.strip().lower()

    # 2. 去除 HTML 标签
    text = re.sub(r'<[^>]+>', '', text)

    # 3. 去除数字
    text = re.sub(r'\d+', '', text)

    # 4. 去除所有中英文标点和特殊符号（保留中英文和空格）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z\s]', ' ', text)

    # 5. 中文停用词（字符级过滤）
    text = ''.join(ch for ch in text if ch not in stop_words)

    # 6. 去除多余空格
    text = re.sub(r'\s+', ' ', text).strip()

    # 7. 去重（保持原顺序）
    words = text.split()
    seen = set()
    unique_words = []
    for w in words:
        if w not in seen:
            seen.add(w)
            unique_words.append(w)

    return ' '.join(unique_words)


In [21]:
df['processed_text'] = df['text'].apply(preprocess_text)

['hello', 'world!', 'this', 'is', 'an', 'example', 'text', 'with', 'numbers', 'and', 'symbols', '#,', '@,', 'and', 'so', 'on.']
["here's", 'another', 'sentence:', 'it', 'contains', 'html', 'tags', 'like', 'and', '&', 'symbols.']
['numbers,', 'punctuations...and', 'stop', 'words', 'are', 'in', 'this', 'sentence!']
['这是一段包含html标签的文本。标签内容此外，文本中还有特殊字符：#、@、$、¥以及一些无用的信息。']
['这是一段含有停用词的文本。停用词包括：和、是、的、了、不等。']
['此文本包含了前后空格以及多余的空白', '间隔。']
['此文本中包含英文标点符号，例如：!', '@', '#', '$，需要对这些符号进行处理。']


In [19]:
print('处理后的文本')
for i, line in enumerate(df['processed_text'].values):
    print(i, line)

处理后的文本
0 hello world this is an example text with numbers and symbols so on
1 heres another sentence it contains html tags like and symbols
2 numbers punctuationsand stop words are in this sentence
3 这是一段包含html标签的文本标签内容此外文本中还有特殊字符以及一些无用的信息
4 这是一段含有停用词的文本停用词包括和是的了不等
5 此文本包含了前后空格以及多余的空白 间隔
6 此文本中包含英文标点符号例如 需要对这些符号进行处理
